<a href="https://colab.research.google.com/github/BuberDev/Machine_Learning/blob/main/PyTorch_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Aby ten notebook działał ze zaktualizowanymi interfejsami API, potrzebujemy torch 1.12+ i torchvision 0.13+try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

In [ ]:
# Kontynujemy z normalnymi importami
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Spróbujemy uzyskać informacje o torch, zainstaluj je, jeśli nie działatry:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Spróbujemy zaimportować katalog going_modular, pobrac go z GitHub, jeśli nie działa
try:
    from going_modular.going_modular import data_setup, engine
except:

    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [ ]:
# Ustawienie urzadzenia
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# 1. Zdobycie danych

In [ ]:
import os
import zipfile

from pathlib import Path

import requests

# Ustawienie sciezki do folderu z danymi
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Jesli flder z obrazami nie istnieje , to pobieramy go i przygotowujemy
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    # Pobranie jedzenia
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Rozpakowanie
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

    # usuniecie .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

In [ ]:
# Ustawienie katalogow
train_dir = image_path / "train"
test_dir = image_path / "test"

# 2. Tworzym zestaw danych i ladowanie danych

In [ ]:
# Tworzymy piipeline transform
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Zmienienie krztaltu  obrazu na  224x224
    transforms.ToTensor(), # 2. Zmiana wartosci obrazu na 0 & 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. Srednia [0.485, 0.456, 0.406] (przez kazdy z kanalow kolorow)
                         std=[0.229, 0.224, 0.225]) # 4. Tzw. standard deviation  [0.229, 0.224, 0.225] (przez kazdy z kanalow kolorow),
])

In [ ]:
# Tworzymy szkoleniowe i testujące DataLoadery, a także uzyskujemy listę nazw klastrain_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms, # zmieniać rozmiar, konwertować obrazy między 0 a 1 i normalizować je
                                                                               batch_size=32) # ustawiamy mini-batch na 32

train_dataloader, test_dataloader, class_names

In [ ]:
# Bierzemy zestaw przetrenowanych wag modelu
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

In [ ]:
# Uzywamy transforms aby stworzyc pre trenowane wagi
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
# Tworzymy trenujace i testujace DataLoaders a także uzyskujemy listę nazw klas
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transforms, # musimy wykonać te same przekształcenia danych na własnych danych, co wstępnie wytrenowany model
                                                                               batch_size=32) # ustawiamy mini-batch 32

train_dataloader, test_dataloader, class_names

# 3. Uzyskujemy pre trenowany model